In [ ]:
# Step 1: Install required packages
!pip install langchain-google-genai langchain faiss-cpu PyPDF2 --quiet
!pip install sentence-transformers --quiet
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66

In [ ]:
# Step 2: Import modules
import os
import PyPDF2
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from google.colab import files
import getpass


In [ ]:
from IPython.display import Markdown, display

In [ ]:
# Step 3: Setup Gemini API key
# Safer way: prompt for key without exposing it
GOOGLE_API_KEY = getpass.getpass("🔑 Enter your Google Gemini API key: ")

llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0,
)

🔑 Enter your Google Gemini API key: ··········


In [ ]:
# Step 4: Setup text splitter and embeddings
text_splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
)

embeddings = HuggingFaceEmbeddings()


<ipython-input-6-280f173e1663>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-6-280f173e1663>:9: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingf

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Step 5: Upload your resume (PDF)
uploaded = files.upload()
pdf_path = next(iter(uploaded))


Saving Resume_Amarnath.pdf to Resume_Amarnath.pdf


In [ ]:
# Step 6: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    return text

resume_text = extract_text_from_pdf(pdf_path)


In [ ]:
# Step 7: Prompt template for summarizing resume
resume_summary_template = """
Role: You are an AI Career Coach.

Task: Given the candidate's resume, provide a comprehensive summary that includes the following key aspects:
- Career Objective
- Skills and Expertise
- Professional Experience
- Educational Background
- Notable Achievements

Instructions:
Provide a concise summary of the resume, focusing on the candidate's skills, experience, and career trajectory. Ensure the summary is well-structured, clear, and highlights the candidate's strengths in alignment with industry standards.

Requirements:
{resume}
"""

resume_prompt = PromptTemplate(
    input_variables=["resume"],
    template=resume_summary_template,
)

resume_analysis_chain = LLMChain(llm=llm, prompt=resume_prompt)
resume_summary = resume_analysis_chain.run(resume=resume_text)

formatted_summary = "### 📄 Resume Summary\n\n" + resume_summary
display(Markdown(formatted_summary))


### 📄 Resume Summary

Okay, here's a summary of Veerabhadra Sai Amarnath Amudalapalli's resume, highlighting key aspects for a potential employer:

**Summary**

Veerabhadra Sai Amarnath Amudalapalli is a highly motivated and skilled Computer Science and Engineering student (AI & ML specialization) at Malla Reddy University, expected to graduate in 2025.  He possesses a strong academic record (CGPA: 9.08) and a demonstrable passion for full-stack web development. Amarnath has experience building complete applications using modern technologies and demonstrates a solid understanding of secure authentication practices and RESTful API design. He is proficient in both front-end and back-end development, showcasing a commitment to continuous learning and practical application of his skills.

**Key Aspects:**

*   **Career Objective:**  Aspiring full-stack developer seeking opportunities to leverage his skills in web development, particularly with Next.js, Node.js, and MongoDB, to contribute to innovative and impactful projects.

*   **Skills and Expertise:**
    *   **Web Development:** Proficient in front-end technologies (HTML, CSS, Tailwind CSS, JavaScript, React) and back-end technologies (Node.js, Express.js, MongoDB, MySQL). Experienced with the Next.js framework for full-stack development.
    *   **Programming Languages:** Competent in Core Java, C++, C, and Python.
    *   **Software Development Practices:** Familiar with version control using Git and GitHub, and experienced with development and debugging tools like VS Code, Chrome DevTools, and Postman.
    *   **Security:** Understands and implements secure authentication methods, including JWT, bcrypt, and email verification.

*   **Professional Experience (Projects):**
    *   **Full Stack Authentication System:** Developed a complete authentication system using Next.js, MongoDB, Nodemailer, and Mailtrap, demonstrating expertise in user management, data protection, and secure session handling.
    *   **VidTube (YouTube-like Backend):** Built a robust backend using Node.js, Express.js, and MongoDB, implementing RESTful APIs for video uploads, playlists, likes, comments, users, and subscriptions. Showcases understanding of database optimization and API design.

*   **Educational Background:**
    *   B.Tech in Computer Science and Engineering (AI & ML) - Malla Reddy University (Expected 2025)
    *   Strong academic performance with a CGPA of 9.08.

*   **Notable Achievements:**
    *   Demonstrated aptitude in programming through winning placements in technical quizzes focused on Python and C.

This summary highlights Amarnath's key strengths and provides a clear picture of his skills and experience to potential employers.

In [ ]:
# Step 8: Build a QA System on Resume
splitted_text = text_splitter.split_text(resume_text)
vectorstore = FAISS.from_texts(splitted_text, embeddings)
vectorstore.save_local("vector_index")

db = FAISS.load_local("vector_index", embeddings, allow_dangerous_deserialization=True)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

career_qa_prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an experienced AI Career Coach.

You are helping a job seeker by answering questions based on their resume and your knowledge.

Here is the relevant context from the candidate's resume:
{context}

Question:
{question}

Give a structured, actionable answer with advice and steps to improve the candidate’s career prospects.

If the resume context is insufficient, use your general career coaching expertise to provide helpful answers.

Response should be straightforward and shorter without unnecessary information.
"""
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": career_qa_prompt_template}
)

In [ ]:
while True:
    query = input("❓ Ask a question about your resume or career (or type 'exit' to quit): ")
    if query.lower() == 'exit':
        break
    response = qa_chain.invoke(query)
    display(Markdown(f"**👤 You:** {query}"))
    display(Markdown(f"### 🧠 AI Career Coach:\n\n{response['result']}"))


❓ Ask a question about your resume or career (or type 'exit' to quit): I want to switch to GEN AI, what should I need to excel apart from My Resume skills


**👤 You:** I want to switch to GEN AI, what should I need to excel apart from My Resume skills

### 🧠 AI Career Coach:

Okay, switching to GenAI from your current full-stack background is a smart move! Here's what you need to excel, building on your existing skills:

**1. Deepen AI/ML Fundamentals:**

*   **Action:** Take online courses (Coursera, deeplearning.ai, fast.ai) focusing on:
    *   **Core ML:** Supervised/Unsupervised learning, Regression, Classification, Clustering.
    *   **Deep Learning:** Neural Networks, CNNs, RNNs, Transformers.
    *   **Natural Language Processing (NLP):** Text processing, Sentiment Analysis, Language Models.
*   **Why:** GenAI is built on these foundations. You need to understand the underlying algorithms.

**2. Focus on GenAI Specific Skills:**

*   **Action:** Learn specific GenAI frameworks and tools:
    *   **TensorFlow/Keras or PyTorch:** Master one of these deep learning frameworks.
    *   **Hugging Face Transformers:** Essential for working with pre-trained language models.
    *   **LangChain:** Framework for developing applications powered by language models.
    *   **Vector Databases (e.g., Pinecone, Chroma):** Learn how to store and retrieve vector embeddings.
*   **Why:** These are the tools used to build and deploy GenAI applications.

**3. Build GenAI Projects:**

*   **Action:** Create projects showcasing your GenAI skills:
    *   **Fine-tune a pre-trained language model:** For text generation, summarization, or translation.
    *   **Build a chatbot:** Using a language model and a conversational framework.
    *   **Implement image generation:** Using models like DALL-E or Stable Diffusion (even if just a basic implementation).
    *   **Contribute to Open Source:** Find GenAI projects on GitHub and contribute.
*   **Why:** Practical experience is crucial. Projects demonstrate your ability to apply your knowledge.

**4. Showcase Your Skills:**

*   **Action:**
    *   **Update your resume:** Highlight your AI/ML skills and GenAI projects.
    *   **Update your GitHub:** Make sure your GenAI projects are well-documented and easy to understand.
    *   **Network:** Attend AI/ML meetups, conferences, and online communities.
    *   **Create Content:** Write blog posts or create videos about your GenAI projects.
*   **Why:** You need to demonstrate your new skills to potential employers.

**5. Tailor Your Learning:**

*   **Action:** Research the specific GenAI roles you're interested in (e.g., prompt engineer, ML engineer, AI researcher) and tailor your learning to those roles.
*   **Why:** Different roles require different skill sets.

**In summary:** You have a solid foundation. Focus on targeted learning, practical projects, and effective showcasing to make a successful transition to GenAI. Good luck!

❓ Ask a question about your resume or career (or type 'exit' to quit): exit
